# Transport Network
This notebook presents a first part of the traditional 4-step transport model, namely the supply side. The following blocks of code create a simple network structure, representing the available transport infrastructure (assuming only one mode for now). Think about the different models presented in the first lecture, coarse transport model and 3-market model. The available network makes it possible to move from one location to another given activities leading to trips. 

From origin to destination the transport network has some impedance. A clear and simple structure to represent such a network is with nodes and links with corresponding attributes giving information about the impedance.

At the end of this part it should be clear:
- how a network is representad and what the key parts are of nodes and links
- what centroids and connectors repsresent (next notebook talks about the demand) 
- how shortest paths between nodes are determined and how the attributes influence the impedance (for the unloaded network)

In order to attain this, some questions are posed throughout the notebooks. These range from very simple questions to more open ended ones, as well as questions to write some code.

- **Q** = normal question
- **Qc** = do something with the code, DIY

## Software

We will use python packages to build graphs and analyze networks. It is important to keep in mind that having a clear and compact data structure makes it easier to integrate it further to come to a full 4-step model, meaning that it is important to automate repetitive processes as much as possible and create an object comprising all information.

In [1]:
# Import packages
import numpy as np
import networkx as nx
from stapy.network_data import *
from stapy.visualization import *
from stapy.settings import config_dict, speed_mapping, cap_mapping, default_capacity, default_speed
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from shapely.geometry import Polygon, box
import pandas as pd
import geopandas as gpd
import geoplot
from math import ceil

### Network
In these notebooks we use a simple theoretical network to show the traditional 4-step model. This makes it easier to understand the basics of transport modelling and helps to go to more complex and real-life examples afterwards.

Let's say we have a fictional urban area called Pyville comprising some road network.

In [7]:
# Initial network Pyville
# A grid-like network is created with corresponding attributes for nodes and links
gr = create_grid_nw(3, 4)
# Some positioning (purely for visualization)  Note: following the example of the Omnitrans tutorial
c_x = np.array([nx.get_node_attributes(gr, 'x')[i] for i in gr.nodes])
c_y = np.array([nx.get_node_attributes(gr, 'y')[i] for i in gr.nodes])
t = np.array([(1, 0.25), (0.8, 0.3), (0.6, 0.25)]) # shift nodes
for e, v in enumerate(list(gr.nodes)):
    if v in [3,4,5,6,7,8]:
        gr.nodes[v]['x'] = c_x[e] + t[1][0]
        gr.nodes[v]['y'] = c_y[e] + t[1][1]
    elif v in [9,10,11]:
        gr.nodes[v]['x'] = c_x[e] + t[2][0]
        gr.nodes[v]['y'] = c_y[e] + t[2][1]
    else:
        gr.nodes[v]['x'] = c_x[e] + t[0][0]
        gr.nodes[v]['y'] = c_y[e] + t[0][1]
        
# val = list(zip(list(nx.get_node_attributes(gr, 'x').values()), list(nx.get_node_attributes(gr, 'y').values())))        
# pos = {v: val[e] for e, v in enumerate(list(gr.nodes.keys()))}

In [3]:
# Visualize the created network of Pyville
plt1 = show_theoretical_network(gr, notebook=True, scaling=0.01, show_plot=True)

Hovering over the nodes and links shows some of the attributes. For links, the capacity, length and travel speed are very important for future steps. For the unloaded network travel times are determined purely on the length and the speed.

- **Q:** What will be the effect if these attribute values change?
- **Q:** How would you incorporate alternative transportation modes?
- **Qc:** Try to list all the attributes for all the links in the network

In [ ]:
# Write code here

### Zones and centroids
The network is ready but where do trips come from? As shortly mentioned in the introduction section, this notebook goes over the supply side, more specificaly the transport infrastructure, but in order to load the network with the travel demand some connection with it should exist. In the traditional modelling setup, demand is aggregated over zones and concentrated in centroids that function as sources and sinks of trips for their corresponding zone. The demand itself is considered in the next notebook about trip generation.

Note that zones can take any shape and can also vary in size, depending on the geography of the study area.
The following block of code defines fictional zones with centroids and is connected to the network of Pyville. 

In [8]:
# Create zones, number of zones is specified in a row-column format 
# with equally sized zones (e.g. meshgrid of rectangles)
zones = create_zones(3, 3, height=1, width=1.5)
centr_list = [(i[1], {'id': i[1],'zone': i[0],'x':i[3], 'y': i[4], 'centroid': True}) for i in zones.values]
gr.add_nodes_from(centr_list)

# Connect centroids to nodes of the graph --> follow omnitrans tutorial example
# c0 -> 0, c1 -> 1, c2 -> 2, c3 -> 6, c4 -> 7, c5 -> 5, c6 -> 9, c7 -> 10, c8 -> 11
# Input of nodes connected to centroids should have same length as number of centroids
gr = connect_centroids_to_graph(gr, [0,1,2,6,7,5,9,10,11])

In [9]:
plt1 = show_theoretical_network(gr, zones=zones, notebook=True, scaling=0.01, show_plot=True)

The figure above shows the network with a centroid for every zone in the study area, in this simple network centroids only have one connector linking them to the network graph, but more may also be possible and depending on the geographic relation and/or the scale of modelling. The actual demand generated is part of the next notebook.

### Shortest Paths
Finding the shortest path between an origin and destination forms the basis for the trip distribution and assignment steps in the traditional 4-step model. The shortest paths can be calculated with different weighting, the default treats all links the same and tries to find the path with minimal number of links to get from one node to the other but also a different weighting is possible and facilitates the inclusion of more variables in the shortest path calculation.

The used package in this notebook has an included shortest path method, the block of code below shows a function to get the path between the nodes in the simple graph. Take a closer look at it.

In [10]:
# Visualize shortest paths in graph based on travel time attribute
def shortest_path(graph, origin=None, destination=None, attribute=None, only_centroids=False):
    if attribute is None:
        attribute='travel_time'
    path = nx.shortest_path(graph, source=origin, target=destination, weight=attribute)
    if destination is None and only_centroids:
        path = {key:path[key] for key in path.keys() if type(key) is str and 'c' in key and key != origin}
    return path

In [11]:
p = shortest_path(gr, 'c1', only_centroids=True)
sgr = gr.subgraph(p['c2'])
show_subgraph(sgr, notebook=True, figure_obj=plt1, show_plot=True, color='red')

Figure(id='1420', ...)

- **Qc:** Try to show another shortest path between two nodes
- **Qc:** Change some of the existing attributes to make some links less interesting. For example, change one edge to a different highway type and recalculate the (unloaded) travel time. Determine the shortest paths with the new settings.

In [ ]:
# Writ code here

In [12]:
import pickle
# Save graph
with open('graph', 'wb') as a:
    pickle.dump(gr, a)
# Save zones
with open('zones', 'wb') as a:
    pickle.dump(zones, a)